In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import time

In [2]:
# Step 1: Real-time data fetcher
def fetch_realtime_data(ticker, interval='1m', period='1d'):
    stock = yf.Ticker(ticker)
    df = stock.history(interval=interval, period=period)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

In [3]:
# Step 2: Feature engineering
def create_features(df):
    df['Returns'] = df['Close'].pct_change()
    df['MA_5'] = df['Close'].rolling(window=5).mean()
    df['STD_5'] = df['Close'].rolling(window=5).std()
    df = df.dropna()
    return df

In [4]:
# Step 3: Train XGBoost model
def train_xgb_model(df):
    features = ['Open', 'High', 'Low', 'Volume', 'MA_5', 'STD_5']
    target = 'Close'

    X = df[features]
    y = df[target]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = XGBRegressor(n_estimators=100, learning_rate=0.1)
    model.fit(X_scaled, y)

    return model, scaler

In [5]:
# Step 4: Predict and decide
def make_decision(model, scaler, df):
    latest = df.iloc[-1:]
    X_pred = latest[['Open', 'High', 'Low', 'Volume', 'MA_5', 'STD_5']]
    X_scaled = scaler.transform(X_pred)
    predicted_price = model.predict(X_scaled)[0]

    current_price = latest['Close'].values[0]

    print(f"🟡 Current Price: {current_price:.2f}")
    print(f"🧠 Predicted Price (Next Minute): {predicted_price:.2f}")

    # Buy/sell/hold logic
    if predicted_price > current_price * 1.01:
        return "✅ BUY"
    elif predicted_price < current_price * 0.99:
        return "❌ SELL"
    else:
        return "🔁 HOLD"

In [6]:
# Step 5: Real-time loop
def run_xgb_realtime_prediction(ticker='AAPL'):
    print(f"📡 Starting XGBoost Stock Predictor for {ticker}...")

    df = fetch_realtime_data(ticker)
    df = create_features(df)
    model, scaler = train_xgb_model(df)

    while True:
        time.sleep(60)  # Fetch every 1 minute
        df = fetch_realtime_data(ticker)
        df = create_features(df)

        try:
            decision = make_decision(model, scaler, df)
            print(f"[{datetime.now().strftime('%H:%M:%S')}] Decision: {decision}")
            print("-" * 50)
        except Exception as e:
            print("⚠️ Error:", e)


In [7]:
run_xgb_realtime_prediction("AAPL")

📡 Starting XGBoost Stock Predictor for AAPL...
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[16:57:30] Decision: 🔁 HOLD
--------------------------------------------------
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[16:58:30] Decision: 🔁 HOLD
--------------------------------------------------
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[16:59:30] Decision: 🔁 HOLD
--------------------------------------------------
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[17:00:30] Decision: 🔁 HOLD
--------------------------------------------------
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[17:01:30] Decision: 🔁 HOLD
--------------------------------------------------
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[17:02:31] Decision: 🔁 HOLD
--------------------------------------------------
🟡 Current Price: 195.34
🧠 Predicted Price (Next Minute): 195.34
[17:03:31] Decision: 🔁 HOLD
---

KeyboardInterrupt: 